In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# -- Chrome'u başlat (istersen headless ile) --
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Tarayıcı arka planda açılsın (görünmez)
options.add_argument("--window-size=1200,800")
driver = webdriver.Chrome(options=options)

# -- Eleman.net iş ilanları sayfasını aç --
url = "https://www.eleman.net/is-ilanlari"
driver.get(url)
time.sleep(5)  # Sayfa ve JS'nin tamamen yüklenmesi için bekle

# -- İş ilanı kartlarını bul --
cards = driver.find_elements(By.CSS_SELECTOR, "div.list-item")

jobs = []
for card in cards:
    try:
        title = card.find_element(By.CSS_SELECTOR, "a.job-title").text.strip()
    except:
        title = None

    try:
        company = card.find_element(By.CSS_SELECTOR, "span.company-name").text.strip()
    except:
        company = None

    try:
        location = card.find_element(By.CSS_SELECTOR, "span.job-location").text.strip()
    except:
        location = None

    try:
        link = card.find_element(By.CSS_SELECTOR, "a.job-title").get_attribute("href")
        if not link.startswith("http"):
            link = "https://www.eleman.net" + link
    except:
        link = None

    jobs.append({
        "Title": title,
        "Company": company,
        "Location": location,
        "Link": link
    })

# -- Sonuçları DataFrame'e çevir --
df = pd.DataFrame(jobs)
print(df.head())

# -- CSV olarak kaydetmek istersen --
df.to_csv("eleman_net_is_ilanlari.csv", index=False)

driver.quit()


Empty DataFrame
Columns: []
Index: []


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()  # Eğer undetected_chromedriver kullanıyorsan burayı değiştir
driver.get("https://www.eleman.net/is-ilanlari")
time.sleep(5)  # Sayfanın yüklenmesini bekle

divs = driver.find_elements(By.TAG_NAME, "div")

for i, div in enumerate(divs):
    cls = div.get_attribute("class")
    txt = div.text.strip()
    if txt:  # Boş olmayanlar
        print(f"--- {i+1}. DIV ---")
        print("CLASS:", cls)
        print("TEXT:", txt[:150])
        print("-" * 40)
    if i > 30:  # Sadece ilk 30 divi incele, fazlası gerekmez
        break

driver.quit()


--- 1. DIV ---
CLASS: c-navbar c-navbar--full ios_uyari
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma Giriş İlan Yayınla
----------------------------------------
--- 2. DIV ---
CLASS: c-navbar__body
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma Giriş İlan Yayınla
----------------------------------------
--- 5. DIV ---
CLASS: c-navbar__right noprint
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma Giriş İlan Yayınla
----------------------------------------
--- 7. DIV ---
CLASS: c-navbar__mobile
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma Giriş İlan Yayınla
----------------------------------------
--- 8. DIV ---
CLASS: c-navbar__tablet
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma Giriş İlan Yayınla
----------------------------------------
--- 9. DIV ---
CLASS: c-navbar__item
TEXT: İş İlanları
----------------------------------------
--- 10. DIV ---
CLASS: c-navbar__item c-dropdown c-dropdo

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.eleman.net/is-ilanlari")
time.sleep(5)

divs = driver.find_elements(By.TAG_NAME, "div")

for i, div in enumerate(divs):
    cls = div.get_attribute("class")
    txt = div.text.strip()
    if txt:
        print(f"--- {i+1}. DIV ---")
        print("CLASS:", cls)
        print("TEXT:", txt[:60])  # İlk 60 karakteri göster, daha hızlı bakabilirsin
        print("-" * 40)
    if i > 100:  # Şimdi ilk 100 divi gösteriyoruz
        break

driver.quit()


--- 1. DIV ---
CLASS: c-navbar c-navbar--full ios_uyari
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma 
----------------------------------------
--- 2. DIV ---
CLASS: c-navbar__body
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma 
----------------------------------------
--- 5. DIV ---
CLASS: c-navbar__right noprint
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma 
----------------------------------------
--- 7. DIV ---
CLASS: c-navbar__mobile
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma 
----------------------------------------
--- 8. DIV ---
CLASS: c-navbar__tablet
TEXT: İş İlanları Araçlar
İş Arayan Girişi Özgeçmiş Oluştur Firma 
----------------------------------------
--- 9. DIV ---
CLASS: c-navbar__item
TEXT: İş İlanları
----------------------------------------
--- 10. DIV ---
CLASS: c-navbar__item c-dropdown c-dropdown--right
TEXT: Araçlar
----------------------------------------
--- 12. DIV ---
CLASS: c-

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, pandas as pd
from bs4 import BeautifulSoup

# 1) Tarayıcıyı başlat
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# options.add_argument("--headless=new")  # görünmez çalıştırmak istersen (ilk denemede kapalı bırak)
driver = webdriver.Chrome(options=options)

try:
    # 2) Sayfaya git ve ana liste kolonunu bekle
    driver.get("https://www.eleman.net/is-ilanlari")
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".l-grid__col--lg-8")))
    time.sleep(2)

    # 3) Lazy load ihtimaline karşı biraz kaydır
    for _ in range(6):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1.2)

    # 4) Sayfa kaynağını al ve soup'a ver
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 5) Detay sayfasına giden linkleri yakala (birkaç olası pattern)
    anchors = soup.select(
        "a[href*='/is-ilani/'], a[href*='/ilan/'], a.job-title"
    )

    jobs, seen = [], set()
    for a in anchors:
        href = a.get("href")
        if not href: 
            continue
        # mutlaklaştır
        if href.startswith("/"):
            href = "https://www.eleman.net" + href
        # aynı linki 2. kez ekleme
        if href in seen:
            continue

        # başlık
        title = (a.get_text(strip=True) or None)

        # en yakın karttan şirket/lokasyon çekmeye çalış
        company = location = None
        card = a.find_parent(["div","li","article","section"])
        if card:
            c = card.select_one(".company-name, [class*='company']")
            if c: company = c.get_text(strip=True)
            l = card.select_one(".job-location, [class*='lokasyon'], [class*='location'], [class*='sehir']")
            if l: location = l.get_text(strip=True)

        jobs.append({"Title": title, "Company": company, "Location": location, "Link": href})
        seen.add(href)

    df = pd.DataFrame(jobs)
    print(f"{len(df)} ilan bulundu")
    display(df.head(10))

    df.to_csv("eleman_net_is_ilanlari.csv", index=False)
finally:
    driver.quit()


30 ilan bulundu


,Title,Company,Location,Link
0,Danışma / Mercedes-Benz Gülsoy Otomotiv / Tuzl...,None,None,https://www.eleman.net/is-ilani/danisma-merced...
1,Teknik Servis Ve Depo PersoneliVizyotek Teknol...,None,None,https://www.eleman.net/is-ilani/teknik-servis-...
2,Kaynak OperatörüGİ GROUP -İzmir - GaziemirGENE...,None,None,https://www.eleman.net/is-ilani/kaynak-operato...
3,Ağır Vasıta-Servis ŞoförüADAMAR DIŞ TİCARET VE...,None,None,https://www.eleman.net/is-ilani/agir-vasita-se...
4,Kurye / Kazan ŞubeYurtiçi Kargo Servisi -Ankar...,None,None,https://www.eleman.net/is-ilani/kurye-kazan-su...
5,Engelli Mağaza Personeli - Kocaeli BölgesiCİVİ...,None,None,https://www.eleman.net/is-ilani/engelli-magaza...
6,Kasiyer Ve Satış Danışmanı - KırklareliTUĞBA K...,None,None,https://www.eleman.net/is-ilani/kasiyer-ve-sat...
7,Kasiyer Ve Satış Danışmanı - AnkaraTUĞBA KURUY...,None,None,https://www.eleman.net/is-ilani/kasiyer-ve-sat...
8,Depo Personeli (Erkek)Efora İnsan Kaynakları -...,None,None,https://www.eleman.net/is-ilani/depo-personeli...
9,Teknik OperatörKLÜH PROFESYONEL HİZMETLER TİC....,None,None,https://www.eleman.net/is-ilani/teknik-operato...


In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, pandas as pd
from bs4 import BeautifulSoup

def first_text(soup, selectors):
    """İlk eşleşen seçicinin temiz metnini döndür."""
    for sel in selectors:
        el = soup.select_one(sel)
        if el:
            txt = el.get_text(strip=True)
            if txt:
                return txt
    return None

def first_attr(soup, selectors, attr):
    for sel in selectors:
        el = soup.select_one(sel)
        if el and el.has_attr(attr):
            val = el.get(attr)
            if val:
                return val.strip()
    return None

# 1) Tarayıcıyı başlat
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# options.add_argument("--headless=new")  # önce kapalı deneyelim
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

try:
    # 2) Liste sayfası
    driver.get("https://www.eleman.net/is-ilanlari")
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".l-grid__col--lg-8")))
    time.sleep(2)

    # Lazy-load olmasın diye biraz kaydır
    for _ in range(6):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(0.8)

    soup_list = BeautifulSoup(driver.page_source, "html.parser")

    # Liste sayfasında linkleri yakala
    anchors = soup_list.select("a[href*='/is-ilani/'], a[href*='/ilan/'], a.job-title")
    seen, jobs = set(), []

    for a in anchors:
        href = a.get("href")
        if not href:
            continue
        if href.startswith("/"):
            href = "https://www.eleman.net" + href
        if href in seen:
            continue
        title = a.get_text(strip=True) or None
        if not title:
            continue
        jobs.append({"Title": title, "Link": href})
        seen.add(href)

    print(f"Liste sayfasından {len(jobs)} link bulundu.")

    # 3) Her ilanın detayına girip Company, Location, Summary çek
    MAX_DETAIL = 30  # istersen artır
    for i, job in enumerate(jobs[:MAX_DETAIL], start=1):
        url = job["Link"]
        try:
            driver.get(url)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(1.2)

            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Başlığı detaydan güncelle (daha temiz olabiliyor)
            title = first_text(soup, [
                "h1", "h1.job-title", ".job-title h1", "header h1",
                "[itemprop='title']", "[itemprop='name']"
            ]) or job["Title"]

            # Company aday seçicileri
            company = first_text(soup, [
                ".company-name", "a.company-name", "span.company-name",
                "[itemprop='hiringOrganization']", ".ilan-firma-adi",
                "a[href*='firma'] .name", "a[href*='/firma/']", "div.company a", "div.company"
            ])

            # Location aday seçicileri
            location = first_text(soup, [
                ".job-location", "li.location", "span.location", ".ilan-sehir",
                "[itemprop='jobLocation'] .locality", "[itemprop='addressLocality']",
                ".detay-bilgi li:contains('Şehir')",  # css4 desteklenmezse çalışmayabilir
            ])
            if not location:
                # bazen breadcrumb veya meta’dan
                location = first_text(soup, [".breadcrumb .active", ".breadcrumb li:last-child"])

            # Summary: meta description > ilk paragraf > kısa kırpılmış açıklama
            summary = first_attr(soup, ["meta[name='description']", "meta[property='og:description']"], "content")
            if not summary:
                summary = first_text(soup, [
                    ".job-description p", ".ilan-aciklama p", ".desc p", ".job-description", ".ilan-aciklama"
                ])
            if summary and len(summary) > 280:
                summary = summary[:277] + "..."

            job["Title"] = title
            job["Company"] = company
            job["Location"] = location
            job["Summary"] = summary

            print(f"[{i}/{min(MAX_DETAIL,len(jobs))}] ✓ {title[:70]}")

            # nazik bekleme
            time.sleep(0.8)

        except Exception as e:
            print(f"[{i}] Detay okunamadı: {url} -> {e}")
            job.setdefault("Company", None)
            job.setdefault("Location", None)
            job.setdefault("Summary", None)
            time.sleep(0.5)

    df = pd.DataFrame(jobs)
    display(df.head(12))
    print(f"Toplam {len(df)} ilan; detay ziyaret edilen: {min(MAX_DETAIL, len(jobs))}")

    df.to_csv("eleman_net_is_ilanlari_detayli.csv", index=False)

finally:
    driver.quit()


Liste sayfasından 30 link bulundu.


/opt/anaconda3/lib/python3.12/site-packages/soupsieve/css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


[1/30] ✓ danışma/Mercedes-Benz Gülsoy Otomotiv/tuzla/bayGKS GROUP
[2/30] ✓ iş ilanları
[3/30] ✓ Kaynak OperatörüGİ GROUP
[4/30] ✓ Ağır Vasıta-Servis ŞoförüADAMAR DIŞ TİCARET VE DENİZCİLİK HİZMETLERİ A
[5/30] ✓ Kurye /Kazan ŞubeYurtiçi Kargo Servisi
[6/30] ✓ Engelli Mağaza Personeli - Kocaeli BölgesiCİVİL MAĞAZACILIK A.Ş
[7/30] ✓ Kasiyer Ve Satış Danışmanı - KırklareliTUĞBA KURUYEMİŞ
[8/30] ✓ Kasiyer Ve Satış Danışmanı - AnkaraTUĞBA KURUYEMİŞ
[9/30] ✓ DEPO PERSONELİ (ERKEK)Efora İnsan Kaynakları
[10/30] ✓ TEKNİK OPERATÖRKLÜH PROFESYONEL HİZMETLER TİC. A.Ş.
[11/30] ✓ Kasiyer Ve Satış Danışmanı - AntalyaTUĞBA KURUYEMİŞ
[12/30] ✓ Kasiyer Ve Satış Danışmanı - UrlaTUĞBA KURUYEMİŞ
[13/30] ✓ Yemek Ve Temizlik ElemanıDtl Elektronik
[14/30] ✓ Sevkiyat PersoneliFirma Adı Gizli
[15/30] ✓ Yazılım Geliştirme Uzmanıİstanbul Alışveriş Merkezleri
[16/30] ✓ Satış Temsilcisiİstanbul Alışveriş Merkezleri
[17/30] ✓ Ön Muhasebe İdari İşler PersoneliYako Groups
[18/30] ✓ Muhasebe UzmanıUranus Hotel
[19/30] ✓

,Title,Link,Company,Location,Summary
0,danışma/Mercedes-Benz Gülsoy Otomotiv/tuzla/ba...,https://www.eleman.net/is-ilani/danisma-merced...,GKS GROUP,None,KATİP ÇELEBİ CADDESİ FATİH MAHALLESİ ÜZERİNDE ...
1,iş ilanları,https://www.eleman.net/is-ilani/teknik-servis-...,None,None,En Yeni iş ilanları eleman.net'te seni bekliyo...
2,Kaynak OperatörüGİ GROUP,https://www.eleman.net/is-ilani/kaynak-operato...,Gİ GROUP,None,"Gi Group, iş gücü piyasasının gelişimi için hi..."
3,Ağır Vasıta-Servis ŞoförüADAMAR DIŞ TİCARET VE...,https://www.eleman.net/is-ilani/agir-vasita-se...,ADAMAR DIŞ TİCARET VE DENİZCİLİK HİZMETLERİ A.Ş.,None,ADAMAR DIŞ TİCARET VE DENİZCİLİK HİZMETLERİ A....
4,Kurye /Kazan ŞubeYurtiçi Kargo Servisi,https://www.eleman.net/is-ilani/kurye-kazan-su...,Yurtiçi Kargo Servisi,None,Yurtiçi Kargo Başkent Bölge Müdürlüğü'ne bağlı...
5,Engelli Mağaza Personeli - Kocaeli BölgesiCİVİ...,https://www.eleman.net/is-ilani/engelli-magaza...,CİVİL MAĞAZACILIK A.Ş,None,"Civil ,1996 yılında İstanbul Kartal Cevizli’de..."
6,Kasiyer Ve Satış Danışmanı - KırklareliTUĞBA K...,https://www.eleman.net/is-ilani/kasiyer-ve-sat...,TUĞBA KURUYEMİŞ,None,KIRKLARELİ Mağazamızda Yetiştirilmek üzere Sat...
7,Kasiyer Ve Satış Danışmanı - AnkaraTUĞBA KURUY...,https://www.eleman.net/is-ilani/kasiyer-ve-sat...,TUĞBA KURUYEMİŞ,None,ANKARA Mağazalarımız da Yetiştirilmek üzere Sa...
8,DEPO PERSONELİ (ERKEK)Efora İnsan Kaynakları,https://www.eleman.net/is-ilani/depo-personeli...,Efora İnsan Kaynakları,None,"Efora İnsan Kaynakları olarak, Depo Personeli ..."
9,TEKNİK OPERATÖRKLÜH PROFESYONEL HİZMETLER TİC....,https://www.eleman.net/is-ilani/teknik-operato...,KLÜH PROFESYONEL HİZMETLER TİC. A.Ş.,None,DANIŞMANLIĞINI YAPMIŞ OLDUĞUMUZ LOJİSTİK DEPOM...


Toplam 30 ilan; detay ziyaret edilen: 30


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time, random

# ================== Ayarlar ==================
BASE_URL      = "https://www.eleman.net/is-ilanlari"
START_PAGE    = 1
MAX_PAGES     = 20          # daha fazla sayfa için yükselt
LIST_WAIT     = 15          # liste sayfası bekleme (saniye)
DETAIL_WAIT   = 15          # detay sayfası bekleme (saniye)
DETAIL_DELAY  = (0.6, 1.2)  # her detay arasında rastgele bekleme aralığı
CSV_PATH      = "eleman_net_is_ilanlari_detayli_paged.csv"
HEADLESS      = False       # True yapmak istersen gizli çalışır (blok riskini artırabilir)
# =============================================

def first_text(soup, selectors):
    for sel in selectors:
        el = soup.select_one(sel)
        if el:
            txt = el.get_text(strip=True)
            if txt:
                return txt
    return None

def first_attr(soup, selectors, attr):
    for sel in selectors:
        el = soup.select_one(sel)
        if el and el.has_attr(attr):
            val = el.get(attr)
            if val:
                return val.strip()
    return None

# ---- Selenium Başlat ----
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
if HEADLESS:
    options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, LIST_WAIT)

all_jobs = []
seen_links = set()

try:
    # ------ SAYFA SAYFA İLERLE ------
    page = START_PAGE
    while page <= MAX_PAGES:
        url = f"{BASE_URL}?sayfa={page}" if page > 1 else BASE_URL
        print(f"\n>>> Sayfa {page}: {url}")
        driver.get(url)
        # liste kolonunu bekle
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".l-grid__col--lg-8")))
        time.sleep(1.2)

        # (opsiyonel) biraz kaydır; bazen ilanın tamamı için
        for _ in range(3):
            driver.execute_script("window.scrollBy(0, 1200);")
            time.sleep(0.5)

        soup_list = BeautifulSoup(driver.page_source, "html.parser")

        # İlan linkleri: birkaç pattern birden
        anchors = soup_list.select("a[href*='/is-ilani/'], a[href*='/ilan/'], a.job-title")
        page_links = []
        for a in anchors:
            href = a.get("href")
            if not href:
                continue
            if href.startswith("/"):
                href = "https://www.eleman.net" + href
            title = a.get_text(strip=True) or None
            if not title:
                continue
            # bazı linkler liste dışı olabilir, filtrele
            if "/is-ilani/" not in href and "/ilan/" not in href:
                continue
            if href in seen_links:
                continue
            page_links.append((title, href))
            seen_links.add(href)

        print(f"Bu sayfada bulunan yeni link sayısı: {len(page_links)}")
        if not page_links:
            # bu sayfada hiç yeni link yoksa muhtemelen bitti
            print("Yeni ilan linki bulunamadı, durduruluyor.")
            break

        # ------ DETAYLARI TOPLA ------
        for i, (title_guess, link) in enumerate(page_links, start=1):
            try:
                driver.get(link)
                WebDriverWait(driver, DETAIL_WAIT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                time.sleep(random.uniform(*DETAIL_DELAY))

                soup = BeautifulSoup(driver.page_source, "html.parser")

                # Title (detaydan daha doğru)
                title = first_text(soup, [
                    "h1", "h1.job-title", ".job-title h1",
                    "[itemprop='title']", "[itemprop='name']"
                ]) or title_guess

                # Company
                company = first_text(soup, [
                    ".company-name", "a.company-name", "span.company-name",
                    "[itemprop='hiringOrganization']", ".ilan-firma-adi",
                    "div.company a", "div.company", "header .company a"
                ])

                # Location
                location = first_text(soup, [
                    ".job-location", "li.location", "span.location",
                    ".ilan-sehir", "[itemprop='jobLocation'] .locality",
                    "[itemprop='addressLocality']",
                ])
                # fallback (breadcrumb'tan şehir çekilebilir)
                if not location:
                    location = first_text(soup, [".breadcrumb li:last-child", ".breadcrumb .active"])

                # Summary (meta description > açıklama ilk paragraf)
                summary = first_attr(soup, [
                    "meta[name='description']",
                    "meta[property='og:description']"
                ], "content")
                if not summary:
                    summary = first_text(soup, [
                        ".job-description p", ".ilan-aciklama p",
                        ".desc p", ".job-description", ".ilan-aciklama"
                    ])
                if summary and len(summary) > 280:
                    summary = summary[:277] + "..."

                all_jobs.append({
                    "Title": title,
                    "Company": company,
                    "Location": location,
                    "Summary": summary,
                    "Link": link,
                    "Page": page
                })

                print(f"  [{i}/{len(page_links)}] ✓ {title[:70]}")

            except Exception as e:
                print(f"  [{i}] Detay okunamadı: {link}  -> {e}")
                all_jobs.append({
                    "Title": title_guess,
                    "Company": None,
                    "Location": None,
                    "Summary": None,
                    "Link": link,
                    "Page": page
                })
                time.sleep(0.4)

        # sonraki sayfa
        page += 1

    df = pd.DataFrame(all_jobs)
    print(f"\nToplam {len(df)} ilan toplandı. Örnek:")
    display(df.head(12))
    df.to_csv(CSV_PATH, index=False)
    print(f"CSV kaydedildi: {CSV_PATH}")

finally:
    driver.quit()



>>> Sayfa 1: https://www.eleman.net/is-ilanlari
Bu sayfada bulunan yeni link sayısı: 30
  [1/30] ✓ danışma/Mercedes-Benz Gülsoy Otomotiv/tuzla/bayGKS GROUP
  [2/30] ✓ Teknik Servis Ve Depo PersoneliVizyotek Teknoloji
  [3/30] ✓ Kaynak OperatörüGİ GROUP
  [4/30] ✓ Ağır Vasıta-Servis ŞoförüADAMAR DIŞ TİCARET VE DENİZCİLİK HİZMETLERİ A
  [5/30] ✓ Kurye /Kazan ŞubeYurtiçi Kargo Servisi
  [6/30] ✓ Engelli Mağaza Personeli - Kocaeli BölgesiCİVİL MAĞAZACILIK A.Ş
  [7/30] ✓ Kasiyer Ve Satış Danışmanı - KırklareliTUĞBA KURUYEMİŞ
  [8/30] ✓ Kasiyer Ve Satış Danışmanı - AnkaraTUĞBA KURUYEMİŞ
  [9/30] ✓ DEPO PERSONELİ (ERKEK)Efora İnsan Kaynakları
  [10/30] ✓ TEKNİK OPERATÖRKLÜH PROFESYONEL HİZMETLER TİC. A.Ş.
  [11/30] ✓ Kasiyer Ve Satış Danışmanı - AntalyaTUĞBA KURUYEMİŞ
  [12/30] ✓ Kasiyer Ve Satış Danışmanı - UrlaTUĞBA KURUYEMİŞ
  [13/30] ✓ Yemek Ve Temizlik ElemanıDtl Elektronik
  [14/30] ✓ Sevkiyat PersoneliFirma Adı Gizli
  [15/30] ✓ Yazılım Geliştirme Uzmanıİstanbul Alışveriş Merkezleri
 

,Title,Company,Location,Summary,Link,Page
0,danışma/Mercedes-Benz Gülsoy Otomotiv/tuzla/ba...,None,None,KATİP ÇELEBİ CADDESİ FATİH MAHALLESİ ÜZERİNDE ...,https://www.eleman.net/is-ilani/danisma-merced...,1
1,Teknik Servis Ve Depo PersoneliVizyotek Teknoloji,None,None,"Vizyotek, Elektronik Güvenlik ve Otomasyon ürü...",https://www.eleman.net/is-ilani/teknik-servis-...,1
2,Kaynak OperatörüGİ GROUP,None,None,"Gi Group, iş gücü piyasasının gelişimi için hi...",https://www.eleman.net/is-ilani/kaynak-operato...,1
3,Ağır Vasıta-Servis ŞoförüADAMAR DIŞ TİCARET VE...,None,None,ADAMAR DIŞ TİCARET VE DENİZCİLİK HİZMETLERİ A....,https://www.eleman.net/is-ilani/agir-vasita-se...,1
4,Kurye /Kazan ŞubeYurtiçi Kargo Servisi,None,None,Yurtiçi Kargo Başkent Bölge Müdürlüğü'ne bağlı...,https://www.eleman.net/is-ilani/kurye-kazan-su...,1
5,Engelli Mağaza Personeli - Kocaeli BölgesiCİVİ...,None,None,"Civil ,1996 yılında İstanbul Kartal Cevizli’de...",https://www.eleman.net/is-ilani/engelli-magaza...,1
6,Kasiyer Ve Satış Danışmanı - KırklareliTUĞBA K...,None,None,KIRKLARELİ Mağazamızda Yetiştirilmek üzere Sat...,https://www.eleman.net/is-ilani/kasiyer-ve-sat...,1
7,Kasiyer Ve Satış Danışmanı - AnkaraTUĞBA KURUY...,None,None,ANKARA Mağazalarımız da Yetiştirilmek üzere Sa...,https://www.eleman.net/is-ilani/kasiyer-ve-sat...,1
8,DEPO PERSONELİ (ERKEK)Efora İnsan Kaynakları,None,None,"Efora İnsan Kaynakları olarak, Depo Personeli ...",https://www.eleman.net/is-ilani/depo-personeli...,1
9,TEKNİK OPERATÖRKLÜH PROFESYONEL HİZMETLER TİC....,None,None,DANIŞMANLIĞINI YAPMIŞ OLDUĞUMUZ LOJİSTİK DEPOM...,https://www.eleman.net/is-ilani/teknik-operato...,1


CSV kaydedildi: eleman_net_is_ilanlari_detayli_paged.csv
